Hi everybody. I thought about sharing a notebook with a method to identify the camera orientation in Sideline view. 
The idea is that if one looks at an asymmetric object, say a PEAR, it will look different from two different views. This difference can be measured by the skewness of the x or y distribution. In a "switched" camera view the skewness along x and along y will switch signs. In  a normal view they will keep the initial sign. One can check whether the perspective view did not mess up the skewness by mutiplying all four skewnesses and check for this product to be larger than zero. If the product is lower than zero, another camera orientation function kicks in based on the slopes of the field lines. 
The known camera orientation, that is, when we have the labels, can be calculated automatically. We just look into the label data and see at frame 4 which team was on the left and which one on the right and compare this info with the tracking data. 
You can see in the dataframe at the end of this notebook the orientation found by the function and the known orientation calculated from the label and tracking data. 
I hope you can include this work in your workflow! Good luck everybody!

The beginnig of this notebook uses the code from our excellent host, Rob https://www.kaggle.com/robikscube/helper-code-helmet-mapping-deepsort

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pylab as plt
import cv2
import os


In [ ]:
#import data
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import cv2
import os

# Read in data files
BASE_DIR = '../input/nfl-health-and-safety-helmet-assignment'

# Labels and sample submission
labels = pd.read_csv(f'{BASE_DIR}/train_labels.csv')
ss = pd.read_csv(f'{BASE_DIR}/sample_submission.csv')

# Player tracking data
tr_tracking = pd.read_csv(f'{BASE_DIR}/train_player_tracking.csv')
te_tracking = pd.read_csv(f'{BASE_DIR}/test_player_tracking.csv')

# Baseline helmet detection labels
tr_helmets = pd.read_csv(f'{BASE_DIR}/train_baseline_helmets.csv')
te_helmets = pd.read_csv(f'{BASE_DIR}/test_baseline_helmets.csv')

# Extra image labels
img_labels = pd.read_csv(f'{BASE_DIR}/image_labels.csv')

In [ ]:
def add_track_features(tracks, fps=59.94, snap_frame=10):
    """
    Add column features helpful for syncing with video data.
    """
    tracks = tracks.copy()
    tracks["game_play"] = (
        tracks["gameKey"].astype("str")
        + "_"
        + tracks["playID"].astype("str").str.zfill(6)
    )
    tracks["time"] = pd.to_datetime(tracks["time"])
    snap_dict = (
        tracks.query('event == "ball_snap"')
        .groupby("game_play")["time"]
        .first()
        .to_dict()
    )
    tracks["snap"] = tracks["game_play"].map(snap_dict)
    tracks["isSnap"] = tracks["snap"] == tracks["time"]
    tracks["team"] = tracks["player"].str[0].replace("H", "Home").replace("V", "Away")
    tracks["snap_offset"] = (tracks["time"] - tracks["snap"]).astype(
        "timedelta64[ms]"
    ) / 1_000
    # Estimated video frame
    tracks["est_frame"] = (
        ((tracks["snap_offset"] * fps) + snap_frame).round().astype("int")
    )
    return tracks


tr_tracking = add_track_features(tr_tracking)
te_tracking = add_track_features(te_tracking)

In [ ]:
# add home and away information on the labels
labels["team"] = labels["label"].str[0].replace("H", "Home").replace("V", "Away")
labels['x'] = labels['left'] + labels['width']/2
labels['y'] = labels['top'] + labels['height']/2
labels['est_frame'] = labels['frame']

#now all frames are in the "est_frame" column

In [ ]:
#now I add columns into the baseline_helmets dataframe so that it accepts the same functions as tr_tracking
sk_helmets = tr_helmets.copy()
sk_helmets['gameKey'] = sk_helmets["video_frame"].str.split("_").str[-4].astype('int64')
sk_helmets['playID'] = sk_helmets["video_frame"].str.split("_").str[-3].astype('int64')
sk_helmets['est_frame']  = sk_helmets["video_frame"].str.split("_").str[-1].astype('int64')
sk_helmets['View']  = sk_helmets["video_frame"].str.split("_").str[-2]
sk_helmets['x'] = sk_helmets['left'] + sk_helmets['width']/2
sk_helmets['y'] = (sk_helmets['top'] + sk_helmets['height']/2)
sk_helmets = sk_helmets.query('conf > 0.4 ')
tr_baseline_helmets = sk_helmets.copy()

In [ ]:
def lines_slope_sgn(img):    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = cv2.blur(gray, (5, 5))
    edges = cv2.Canny(gray,50,150,apertureSize = 3)
    MLL = 200
    MLG = 10
    lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength=MLL, maxLineGap=MLG)
    slopes = []
    for i in range(0,len(lines)):
        slope = (lines[i][0][3] - lines[i][0][1])/ (lines[i][0][2] - lines[i][0][0]+0.0001)
        slopes.append(slope)
    slopes_nda = np.array(slopes)
    slopes_sgn = 1*(slopes_nda > 0) - 1*(slopes_nda < 0)
    slopes_sgn = slopes_sgn.sum()
    return slopes_sgn


In [ ]:
def camera_orientation_img(tracking_df, img):
    x_cm = tracking_df['x'].mean()
    if (x_cm - 60)*lines_slope_sgn(img) > 0:
        orientation = "OK"
        return orientation
    else:
        orientation = "switched"
        return orientation
    return False  

In [ ]:
def camera_orientation(helmets_df, tracking_df,img):
    helmets_df = helmets_df.reset_index(drop=True)
    tracking_df = tracking_df.reset_index(drop=True)
    helmets_df['y'] = -helmets_df['y']
    if len(helmets_df) > 22:
        helmets_df = helmets_df.sort_values(by = 'conf', ascending=False).head(22)
        return camera_orientation_img(tracking_df, img)
    sk_x_tr = tracking_df['x'].skew()
    sk_y_tr = tracking_df['y'].skew()
    sk_x_he = helmets_df['x'].skew()
    sk_y_he = helmets_df['y'].skew()
    
    if sk_x_tr*sk_y_tr*sk_x_he*sk_y_he > 0:
        if sk_x_tr*sk_x_he >0:
            orientation = "OK"
            return orientation
        else: 
            orientation = "switched"
            return orientation

    
    return camera_orientation_img(tracking_df, img)
 

In [ ]:
def known_camera_orientation_s(labels_df, tracking_df):
    labels_H = labels_df.query('team == "Home"')
    #labels_H = labels_H.reset_index(drop=True)
    labels_V = labels_df.query('team == "Away"')
    tracking_H = tracking_df.query('team == "Home"')
    tracking_V = tracking_df.query('team == "Away"')
    xh_labels_cm = labels_H['x'].mean()
    xv_labels_cm = labels_V['x'].mean()
    xh_tracking_cm = tracking_H['x'].mean()
    xv_tracking_cm = tracking_V['x'].mean()
    if (xh_labels_cm-xv_labels_cm)*(xh_tracking_cm-xv_tracking_cm)>0:
        orientation = "OK"
    else: 
        orientation = "switched"
    return orientation

In [ ]:

df = pd.read_csv("../input/nfl-health-and-safety-helmet-assignment/train_labels.csv")

files = files = df.video.unique().tolist()
df_files = pd.DataFrame(files)
df_files  = df_files.rename(columns={0:"video_frame"})
df_files['gameKey']  = df_files["video_frame"].str.split("_").str[-3].astype('int64')
df_files['playID'] = df_files["video_frame"].str.split("_").str[-2].astype('int64')
df_files['View1']  = df_files["video_frame"].str.split("_").str[-1]
df_files['View']  = df_files["View1"].str.split(".").str[-2]
df_sideline_videos = df_files.query('View == "Sideline"')
df_sideline_videos = df_sideline_videos.reset_index(drop=True)


In [ ]:
for k in range(0,len(df_sideline_videos)):
    video_name = df_sideline_videos["gameKey"][k]
    play_name = df_sideline_videos['playID'][k]
    find_play_tracking = tr_tracking.query('gameKey == @video_name & est_frame == 4 & playID == @play_name')
    find_play_helmets = tr_baseline_helmets.query('gameKey == @video_name & est_frame == 4 & playID == @play_name & View == "Sideline"')
    find_play_labels = labels.query('gameKey == @video_name & frame == 4 & playID == @play_name')
    find_play_tracking = find_play_tracking.reset_index(drop=True)
    find_play_helmets = find_play_helmets.reset_index(drop=True)
    find_play_labels = find_play_labels.reset_index(drop=True)
    
    path = '../input/nfl-health-and-safety-helmet-assignment/train'
    source = os.path.join(path, df_sideline_videos['video_frame'][k])
    
    cap = cv2.VideoCapture(source)
    for i in range(0,4):
        ret,img = cap.read()
    
    df_sideline_videos.at[k,'camera_view'] =  camera_orientation(find_play_helmets, find_play_tracking,img)
    df_sideline_videos.at[k,'known_camera_view'] = known_camera_orientation_s(find_play_labels, find_play_tracking)
    df_sideline_videos.at[k,'number of helmets'] = len(find_play_helmets)
    df_sideline_videos['agreement'] = df_sideline_videos['camera_view'] == df_sideline_videos['known_camera_view']
df_sideline_videos   